# Call Steering

En este cuaderno abordaremos el problema de enrutamiento de llamadas como un problema de clasficación. Para mayor flexibilidad de uso, los métodos principales se encuentran en un fichero Python corriente, por lo que puede importarse en otras aplicaciones o cuadernos.


En primer lugar, importaremos todos las funciones y objetos auxiliares.

In [13]:
from call import *
from sklearn.svm import LinearSVC

A continuación, obtenemos los datos de entrenamiento.

In [12]:
text,target,classes,classes_reverse=recover_from_files(design_filepath)

# Convert class to numeric
y_data=[classes_reverse[x] for x in target]

Definiremos un par de funciones auxiliares. 

La primera construye una tubería que recibe los datos y realiza automáticamente el procesamiento. En concreto, se utilizará un modelo basado en "bag of words", por lo que se pierde la información sobre el orden de la sentencia. El uso de tuberías permite la búsqueda de hiperparámetros para el preprocesado. Por ejemplo, si es mejor utilizar stop words o no.

La segunda función compara dos clasificadores y devuelve el mejor respecto a la precisión media conseguida por cada clasificador.

In [ ]:
#Al principio, no tenemos mejor clasificador
best=None

In [3]:
def construct_pipe(clf,params):
    """
    Dado un clasificador y sus parámetros, construye una tubería que procesa los datos
    """
    prefix="clf__"
    parameters={'counter__stop_words':[None,'english'],
           'tf_idf__use_idf':[True,False]}
    if type(params) is dict:
        for k in params:
            #Add hyperparameters to dict
            parameters[prefix+k]=params[k]
    elif type(params) is list:
        par=parameters
        parameters=[]
        for line in params:
            aux_params={}
            aux_params.update(par)
            for k in line:
                #Add hyperparameters to dict
                aux_params[prefix+k]=line[k]
            parameters.append(aux_params)
    #Create pipeline
    count_vect = StemmedCountVectorizer()

    tf_transformer = TfidfTransformer()
    #svd=TruncatedSVD(n_components=300)

    pipe=Pipeline([('counter',count_vect),
        ('tf_idf',tf_transformer),
        ('clf',clf)
    #    ('svd',svd)
        ])
    
    return pipe, parameters


def compare_best_classifier(clf1,clf2):
    """
    Dados dos clasificadores entrenados con GridSearch, devuelve el que tiene la mejor tasa de aciertos media
    """
    val_metric='mean_test_acc'
    if clf1 is None:
        return clf2
    return clf1 if clf1.cv_results_[val_metric][clf1.best_index_]>clf2.cv_results_[val_metric][clf2.best_index_] else clf2

## Clasificadores

Ya podemos entrenar los clasificadores. Comenzaremos por Naive Bayes, pues suele utilizarse como clasificador base para comparar el rendimiento, dada su simplicidad a la par que efectividad.

In [4]:
parameters =  {'alpha': [0.01, 1, 5]}
clf=MultinomialNB()
pipe,params=construct_pipe(clf,parameters)
estimator=busqueda_cv(pipe,text,y_data,params)
best=compare_best_classifier(best,estimator)
print(estimator.best_params_ )

Pipeline(memory=None,
     steps=[('counter', StemmedCountVectorizer(analyzer='word', binary=False, decode_error='strict',
            dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
            lowercase=True, max_df=1.0, max_features=None, min_df=1,
            ngram_range=(1, 1), preprocessor=None, stop_wor...lse,
         use_idf=False)), ('clf', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))])
Train acc 0.9756927059465916
Train prec 0.9759217720903675
Train rec 0.9756927059465916
Train f1 0.9755286963157636
Test acc 0.796887159533074
Test prec 0.8080580587879579
Test rec 0.796887159533074
Test f1 0.7918626322691907
{'clf__alpha': 0.01, 'counter__stop_words': None, 'tf_idf__use_idf': False}


Las métricas de validación medias rondan el 79%, lo que no está mal. Curiosamente, la mejor combinación de parámetros es no usar stop words ni idf para tener en cuenta la rareza de los términos en los documentos.

El siguiente clasificador que vamos a utilizar es regresión logística.


In [5]:
parameters =  {'penalty':('l1','l2'),'C': [0.1, 1, 10]}
clf = LogisticRegression()
pipe,params=construct_pipe(clf,parameters)
estimator=busqueda_cv(pipe,text,y_data,params)
best=compare_best_classifier(best,estimator)
print(estimator.best_params_ )

Pipeline(memory=None,
     steps=[('counter', StemmedCountVectorizer(analyzer='word', binary=False, decode_error='strict',
            dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
            lowercase=True, max_df=1.0, max_features=None, min_df=1,
            ngram_range=(1, 1), preprocessor=None, stop_wor...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])
Train acc 0.9869732892719216
Train prec 0.9871680304503782
Train rec 0.9869732892719216
Train f1 0.9869425132763509
Test acc 0.8809338521400778
Test prec 0.8868755181358511
Test rec 0.8809338521400778
Test f1 0.879476739178115
{'clf__C': 10, 'clf__penalty': 'l2', 'counter__stop_words': None, 'tf_idf__use_idf': False}


Como era de esperar, el rendimiento conseguido es sensiblemente superior al de Naive Bayes. De nuevo, no se utilizan stop words Nos quedamos con este como el mejor hasta ahora.

Probaremos ahora con máquinas de vectores soporte (SVM). Para estos hay que ajustar el kernel y sus parámetros, por lo que probaremos con varios de ellos.

In [6]:
parameters = [
  {'C': [0.1, 1, 10], 'kernel': ['linear']},
  {'C': [0.1, 1, 10], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
clf = SVC()
pipe,params=construct_pipe(clf,parameters)
estimator=busqueda_cv(pipe,text,y_data,params)
best=compare_best_classifier(best,estimator)
print(estimator.best_params_ )

Pipeline(memory=None,
     steps=[('counter', StemmedCountVectorizer(analyzer='word', binary=False, decode_error='strict',
            dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
            lowercase=True, max_df=1.0, max_features=None, min_df=1,
            ngram_range=(1, 1), preprocessor=None, stop_wor...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
Train acc 0.9768577307534259
Train prec 0.9777753732532293
Train rec 0.9768577307534259
Train f1 0.9768362095903804
Test acc 0.8856031128404669
Test prec 0.8950015376546079
Test rec 0.8856031128404669
Test f1 0.8846482078694289
{'clf__C': 1, 'clf__kernel': 'linear', 'counter__stop_words': None, 'tf_idf__use_idf': True}


Observamos que conseguimos algo más de rendimiento con este clasificador. También mejoramos otras métricas como la precisión y el f1.

Como el mejor kernel ha resultado ser 'linear', podemos probar con una implementación más eficiente para explorar más parámetros y probar a obtener un mejor resultado.

In [16]:
parameters = {'C': [0.1, 1, 10],'tol':[1e-3,1e-4,1e-5],'class_weight':[None,'balanced']}
 
clf = LinearSVC()
pipe,params=construct_pipe(clf,parameters)
estimator=busqueda_cv(pipe,text,y_data,params)
best=compare_best_classifier(best,estimator)
print(estimator.best_params_ )

Pipeline(memory=None,
     steps=[('counter', StemmedCountVectorizer(analyzer='word', binary=False, decode_error='strict',
            dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
            lowercase=True, max_df=1.0, max_features=None, min_df=1,
            ngram_range=(1, 1), preprocessor=None, stop_wor...max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.001,
     verbose=0))])
Train acc 0.9871595358581313
Train prec 0.9875053300884554
Train rec 0.9871595358581313
Train f1 0.9871593112778706
Test acc 0.8863813229571984
Test prec 0.8951286926001604
Test rec 0.8863813229571984
Test f1 0.8847856310713369
{'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__tol': 0.001, 'counter__stop_words': None, 'tf_idf__use_idf': False}


La mejora ha sido muy pequeña, pero suficiente como para quedarnos con esta.


El próximo clasificador utiliza descenso por gradiente estocástico. Por lo general, esto le permite escapar de mínimos locales para conseguir mejores resultados. Entrenaremos tanto un SVM como un regresor logístico.

In [24]:
parameters={'max_iter': [1000] ,'tol':[1e-2,1e-3,1e-4],'penalty':['l2','l1','elasticnet'],
    'loss':('hinge','log'), 'alpha':[0.0001,0.001]}
clf=SGDClassifier()
pipe,params=construct_pipe(clf,parameters)
estimator=busqueda_cv(pipe,text,y_data,params)
best=compare_best_classifier(best,estimator)
print(estimator.best_params_ )

Pipeline(memory=None,
     steps=[('counter', StemmedCountVectorizer(analyzer='word', binary=False, decode_error='strict',
            dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
            lowercase=True, max_df=1.0, max_features=None, min_df=1,
            ngram_range=(1, 1), preprocessor=None, stop_wor...et', power_t=0.5, random_state=None,
       shuffle=True, tol=0.0001, verbose=0, warm_start=False))])
Train acc 0.9873575043573032
Train prec 0.9875341071831795
Train rec 0.9873575043573032
Train f1 0.9873440637394835
Test acc 0.8832684824902723
Test prec 0.8913748356083889
Test rec 0.8832684824902723
Test f1 0.8813688940664878
{'clf__alpha': 0.0001, 'clf__loss': 'log', 'clf__max_iter': 1000, 'clf__penalty': 'elasticnet', 'clf__tol': 0.0001, 'counter__stop_words': None, 'tf_idf__use_idf': False}


También consigue un rendimiento alto, aunque se queda ligeramente por debajo del clasificador anterior.

Por último, vamos a probar con árboles de decisión. Utilizaremos un árbol estándar y luego la variante random forest.

In [23]:
parameters={'min_samples_split':[0.0001,0.001,0.01],'criterion':('gini','entropy')}
clf=DecisionTreeClassifier()
pipe,params=construct_pipe(clf,parameters)
estimator=busqueda_cv(pipe,text,y_data,params)
best=compare_best_classifier(best,estimator)
print(estimator.best_params_ )

Pipeline(memory=None,
     steps=[('counter', StemmedCountVectorizer(analyzer='word', binary=False, decode_error='strict',
            dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
            lowercase=True, max_df=1.0, max_features=None, min_df=1,
            ngram_range=(1, 1), preprocessor=None, stop_wor...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])
Train acc 0.9929953308403136
Train prec 0.9932892721304407
Train rec 0.9929953308403136
Train f1 0.9930141515998896
Test acc 0.7844357976653696
Test prec 0.8069069528120919
Test rec 0.7844357976653696
Test f1 0.7843418419610835
{'clf__criterion': 'gini', 'clf__min_samples_split': 0.0001, 'counter__stop_words': 'english', 'tf_idf__use_idf': False}


In [22]:
parameters={'n_estimators':[5,10], 'criterion':('gini','entropy'),'min_samples_split':[0.0001,0.001,0.01]}
clf= RandomForestClassifier()
pipe,params=construct_pipe(clf,parameters)
estimator=busqueda_cv(pipe,text,y_data,params)
best=compare_best_classifier(best,estimator)
print(estimator.best_params_ )

Pipeline(memory=None,
     steps=[('counter', StemmedCountVectorizer(analyzer='word', binary=False, decode_error='strict',
            dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
            lowercase=True, max_df=1.0, max_features=None, min_df=1,
            ngram_range=(1, 1), preprocessor=None, stop_wor...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
Train acc 0.9883192993734117
Train prec 0.9885866218875702
Train rec 0.9883192993734117
Train f1 0.9883146241349395
Test acc 0.8171206225680934
Test prec 0.8330632928828049
Test rec 0.8171206225680934
Test f1 0.816567594597479
{'clf__criterion': 'gini', 'clf__min_samples_split': 0.0001, 'clf__n_estimators': 10, 'counter__stop_words': 'english', 'tf_idf__use_idf': True}


Por desgracia, los árboles de decisión no clasifican tan bien en este caso como los SVM o la regresión logística. El mejor de todos ha sido el LinearSVC. Con este último haremos la evaluación final con los datos de prueba.

In [25]:
def rendimiento_test(clf):
    """
    Dado un clasificador, obtiene su rendimiento con un conjunto de prueba
    """
    text,target,classes,classes_reverse=recover_from_files(eval_filepath)

    # Convert class to numeric
    y_test=[classes_reverse[x] for x in target]

    #Vectorize data

    print("Final test")
    for key in scoring:
        print(key,scoring[key](clf,text,y_test))

In [27]:
rendimiento_test(best)
print(best.best_params_)

Final test
acc 0.8807947019867549
prec 0.8887261928652658
rec 0.8807947019867549
f1 0.881159789437935
{'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__tol': 0.001, 'counter__stop_words': None, 'tf_idf__use_idf': False}


El rendimiento final es de un 88%. Para las cuatro métricas se consigue un valor similar, y además no se aleja de los datos de validación del clasificador. Por tanto, podemos concluir que el clasificador es considerablemente bueno.

In [4]:
import csv
glove_path="../glove.6B/glove.6B.50d.txt"
w2v={}
with open(glove_path, encoding="utf-8") as file:
    for line in file:
        splits=line.strip().split(" ")
        w2v[splits[0]]=splits[1:]

In [33]:
#Create pipeline
count_vect = StemmedCountVectorizer()

tf_transformer = TfidfTransformer()
pipe=Pipeline([('counter',count_vect),
    ('tf_idf',tf_transformer)  ])
text_vectorized=pipe.fit_transform(text)
unit_vector=pipe.steps[1][1].transform(np.ones((text_vectorized.shape[1]))).toarray()[0]
print(unit_vector)
def idf(word):
    # get vocabulary index
    idx=pipe.steps[0][1].vocabulary_.get(word,-1)
    res=min(unit_vector)
    if idx>-1:
        res=unit_vector[idx]
#     else:
#         print(word)
    return res

[0.04502558 0.03595504 0.03949976 0.04502558 0.04502558 0.04502558
 0.04258036 0.04502558 0.04502558 0.04502558 0.04258036 0.04502558
 0.04084546 0.03666534 0.04502558 0.02869432 0.03949976 0.01382718
 0.04502558 0.02638463 0.04502558 0.04084546 0.03666534 0.04502558
 0.04502558 0.04502558 0.04502558 0.03840025 0.04502558 0.04084546
 0.0271121  0.04502558 0.04502558 0.04502558 0.04502558 0.04502558
 0.04502558 0.04502558 0.03595504 0.04502558 0.03747062 0.03949976
 0.03949976 0.04502558 0.03113953 0.03949976 0.02611656 0.04258036
 0.04502558 0.04502558 0.04502558 0.04502558 0.04084546 0.03666534
 0.04258036 0.04502558 0.03747062 0.01902873 0.04258036 0.04502558
 0.04502558 0.04502558 0.04502558 0.02493027 0.04502558 0.04502558
 0.04502558 0.04502558 0.04502558 0.04502558 0.04502558 0.04502558
 0.04258036 0.04084546 0.03144886 0.04502558 0.0254943  0.03840025
 0.0156945  0.04502558 0.04084546 0.03329051 0.04258036 0.04502558
 0.03949976 0.04502558 0.04502558 0.02811954 0.04084546 0.0366

In [34]:
import nltk
def word2vec(text):
    dim=[0]*len(w2v[","])
    return np.array([w2v.get(stemmer.stem(word.lower()),dim) for word in nltk.word_tokenize(text)],dtype=float)

def idf_vec(text):
    w2vec=word2vec(text)
    
    for i,word in enumerate(nltk.word_tokenize(text)):
        w2vec[:,i]=w2vec[:,i]*idf(word.lower()) 
    return w2vec

In [35]:
x=word2vec("\"hello I am yopoe")
print(x)

[[ 1.2817e-01  1.5858e-01 -3.8843e-01 -3.9108e-01  6.8366e-01  8.1259e-04
  -2.2981e-01 -6.3358e-01 -2.7663e-01  4.0934e-01 -6.5128e-01  8.4610e-01
  -9.9040e-01  2.0696e-01  1.2567e+00  6.4774e-02  6.5813e-01  3.9954e-01
   7.6104e-02 -5.4083e-01 -3.2438e-01  8.4560e-01  1.7273e-01 -1.3504e-01
   3.9626e-01 -2.3358e+00 -1.6576e+00  5.9957e-01  1.0876e+00 -1.0118e+00
   3.3300e+00  7.5853e-02 -6.5637e-01 -1.5799e-02 -8.5429e-01 -4.7358e-01
   8.2404e-02 -6.9719e-01  4.6647e-01 -3.2044e-01 -4.5517e-01  3.0804e-01
   7.5020e-02 -2.1783e-02  1.0823e-01 -3.3060e-02 -2.5140e-01  8.8184e-02
  -2.2215e-01  1.4971e+00]
 [-3.8497e-01  8.0092e-01  6.4106e-02 -2.8355e-01 -2.6759e-02 -3.4532e-01
  -6.4253e-01 -1.1729e-01 -3.3257e-01  5.5243e-01 -8.7813e-02  9.0350e-01
   4.7102e-01  5.6657e-01  6.9850e-01 -3.5229e-01 -8.6542e-01  9.0573e-01
   3.5760e-02 -7.1705e-02 -1.2327e-01  5.4923e-01  4.7005e-01  3.5572e-01
   1.2611e+00 -6.7581e-01 -9.4983e-01  6.8666e-01  3.8710e-01 -1.3492e+00
   6.3512e-

In [36]:
x=idf_vec("\"hello I am yopoe")
print(x)

[[ 1.77222941e-03  4.40292509e-03 -5.37089077e-03 -5.40753279e-03
   9.45308855e-03  8.12590000e-04 -2.29810000e-01 -6.33580000e-01
  -2.76630000e-01  4.09340000e-01 -6.51280000e-01  8.46100000e-01
  -9.90400000e-01  2.06960000e-01  1.25670000e+00  6.47740000e-02
   6.58130000e-01  3.99540000e-01  7.61040000e-02 -5.40830000e-01
  -3.24380000e-01  8.45600000e-01  1.72730000e-01 -1.35040000e-01
   3.96260000e-01 -2.33580000e+00 -1.65760000e+00  5.99570000e-01
   1.08760000e+00 -1.01180000e+00  3.33000000e+00  7.58530000e-02
  -6.56370000e-01 -1.57990000e-02 -8.54290000e-01 -4.73580000e-01
   8.24040000e-02 -6.97190000e-01  4.66470000e-01 -3.20440000e-01
  -4.55170000e-01  3.08040000e-01  7.50200000e-02 -2.17830000e-02
   1.08230000e-01 -3.30600000e-02 -2.51400000e-01  8.81840000e-02
  -2.22150000e-01  1.49710000e+00]
 [-5.32304873e-03  2.22372983e-02  8.86405076e-04 -3.92069634e-03
  -3.70001457e-04 -3.45320000e-01 -6.42530000e-01 -1.17290000e-01
  -3.32570000e-01  5.52430000e-01 -8.7813

In [37]:
np.mean(x,axis=0)

array([ 5.77284684e-04,  8.38549271e-03,  2.30369083e-04, -5.33806505e-03,
        5.42526200e-03, -4.05657482e-01, -1.72272000e-01,  1.00786000e-01,
       -6.18836000e-01,  1.53175514e-01, -2.03166600e-01,  4.42054000e-01,
       -4.44954000e-01,  5.95444000e-02,  8.66380000e-01,  2.20908800e-01,
       -5.10780000e-02,  6.15910000e-01, -1.36458000e-01, -1.87395000e-01,
       -2.24473600e-01,  6.75792000e-01,  4.92828000e-01,  3.81318000e-01,
        9.09112000e-01, -1.41094200e+00, -8.61940000e-01,  4.12130000e-01,
        6.77682000e-01, -6.18324000e-01,  1.99738400e+00,  2.96618600e-01,
       -4.96160000e-01,  2.68820200e-01, -5.66234000e-01, -4.60728000e-01,
        4.19234800e-01, -2.25156000e-01,  2.23725400e-01, -2.77274000e-01,
        1.03317140e-01, -2.04143600e-01, -1.31096000e-01,  1.09539400e-01,
        1.01731200e-01,  1.12204000e-01, -1.86606200e-01, -1.95380000e-01,
       -1.99314000e-02,  8.89830000e-01])

In [38]:
x=[idf_vec(line) for line in text]
for mat,line in zip(x,text):
    for i,word in enumerate(mat):
        if (mat==0).all():
            print(mat)
            print(line.split()[i])
            
x_transformed=[np.mean(line,axis=0) for line in x]

In [39]:
parameters =  {'penalty':('l1','l2'),'C': [0.1, 1, 10]}
clf = LogisticRegression()
estimator=busqueda_cv(clf,x_transformed,y_data,parameters)
best=compare_best_classifier(best,estimator)
print(best.best_params_ )

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Train acc 0.8643665027789588
Train prec 0.8635565982483968
Train rec 0.8643665027789588
Train f1 0.8627645624408583
Test acc 0.6949416342412451
Test prec 0.7083793888096932
Test rec 0.6949416342412451
Test f1 0.6866830210018576
{'C': 10, 'kernel': 'linear'}


In [40]:
parameters = [
  {'C': [0.1, 1, 10], 'kernel': ['linear']},
  {'C': [0.1, 1, 10], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
clf = SVC()
estimator=busqueda_cv(clf,x_transformed,y_data,parameters)
best=compare_best_classifier(best,estimator)
print(best.best_params_ )

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Train acc 0.9190453815629187
Train prec 0.9200924859323696
Train rec 0.9190453815629187
Train f1 0.9189740531517323
Test acc 0.7042801556420234
Test prec 0.7205827885876445
Test rec 0.7042801556420234
Test f1 0.699956429487653
{'C': 10, 'kernel': 'linear'}
